# Dynamic Programming

Algorithms that can compute optimal policies (solve RL problems) given a perfect model of the environment as an MDP. Computationally, we require a finite MDP and a small enough problem that we can actually solve it in a reasonable time; this is often called the "tabular" case.

The Bellman optimality equations are
$$ v_*\left(s\right) = \max_a \sum_{s',r} p\left(s',r\middle|s,a\right) \left[r + \gamma v_*\left(s'\right)\right] $$
and
$$ q_*\left(s,a\right) = \sum_{s',r} p\left(s',r\middle|s,a\right) \left[r + \gamma \max_{a'} q_*\left(s', a'\right)\right] $$

RL DP algorithms are derived by converting these into update formulas that can be used to iteratively improve an estimate of the value functions.

# Policy Evaluation / Prediction

Given a policy $\pi$, we would like to compute $v_{\pi}\left(s\right) \forall s \in \mathcal{S}$. We have seen earlier that this is a linear system of equations, but it is often tedious to set up and solve the system. The **iterative policy evaluation algorithm** estimates it via an update formula:

$$ \begin{align*} v_{k+1}\left(s\right) & = \mathbb{E}_{\pi}\left[R_{t+1} + \gamma v_k \left(S_{t+1}\right) \middle| S_t = s \right] \\ & = \sum_a \pi\left(a\middle|s\right) \sum_{s',r} p\left(s',r\middle| s,a\right) \left[r + \gamma v_k\left(s'\right) \right]\end{align*} $$

This looks nearly identical to the Bellman equation for $v_{\pi}$, but the crucial difference is that $v_{k+1}$ on the left-hand side and $v_k$ on the right are not the same function; the former is an incremental update of the latter.

First we will set up the environment/dynamics of the chapter 4 gridworld.

In [ ]:
# 0: up, 1: right, 2: down, 3: left
def dynamics(s, a):
    """Dynamics for the episodic gridworld environment.
    Given (s,a), return a list of (s', r, p), possible outcomes.
    """
    if s == 0 or s == 15: # terminal states
        return [(0, 0, 1.0)]
    # else we are not in the terminal states

    if a == 0: # moving up
        if s < 4: # hit top boundary, stay in place
            return [
                (s, -1, 1.0),
            ]
        else: # we are not at the top row
            return [
                (s - 4, -1, 1.0)
            ]
    elif a == 1: # moving right
        if (s+1) % 4 == 0: # hit right boundary
            return [
                (s, -1, 1.0)
            ]
        else: # we are not in the right row
            return [
                (s+1, -1, 1.0),
                ]
        
    elif a == 2: # moving down
        if s >= 12: # hit bottom boundary
            return [
                (s, -1, 1.0)
                ]
        else: # we are not in the bottom row
            return [
                (s + 4, -1, 1.0)
                ]
        
    elif a == 3: # moving left
        if s % 4 == 0: # hit left boundary
            return [
                (s, -1, 1.0)
            ]
        else: # not in leftmost row
            return [
                (s - 1, -1, 1.0)
            ]

In [ ]:
from typing import Callable

def plot_gridworld_policy(policy: Callable, ax = None):
    if ax is None:
        _, ax = plt.subplots()

    ax.set(xlim=[-0.5,3.5], ylim=[-0.5,3.5])
    ax.set_xticks(np.arange(-0.5, 3.5, 1))
    ax.set_yticks(np.arange(-0.5, 3.5, 1))
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.grid()

    def state_to_txt_coord(s: int):
        y_coord = 4 - ( s // 4) - 1
        x_coord = s % 4
        return x_coord, y_coord

    arrows = {0: [0,1], 1: [1, 0], 2: [0, -1], 3: [-1, 0]}
    scale = 0.40
    eps = 1e-5
    for s in range(16):
        x_t, y_t = state_to_txt_coord(s)
        for a, p_a_s in policy(s):
            arrow = np.array(arrows[a])
            arrow = scale * p_a_s * arrow
            if any(abs(arrow) > eps):
                ax.arrow(
                    x_t,
                    y_t,
                    arrow[0],
                    arrow[1],
                    head_width=0.05,
                    head_length=0.1,
                    fc='k',
                    ec='k',
                    )
    ax.set_aspect('equal')
    # plt.show()

def plot_v(v: np.ndarray, ax = None):
    if ax is None:
        _, ax = plt.subplots()

    ax.set(xlim=[-0.5,3.5], ylim=[-0.5,3.5])
    ax.set_xticks(np.arange(-0.5, 3.5, 1))
    ax.set_yticks(np.arange(-0.5, 3.5, 1))
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.grid()

    
    def state_to_txt_coord(s: int):
        y_coord = 4 - ( s // 4) - 1
        x_coord = s % 4
        return x_coord, y_coord

    for s in range(len(v)):
        x_t, y_t = state_to_txt_coord(s)
        ax.text(x=x_t, y=y_t, s=f"{v[s]:.1f}")
    ax.set_aspect('equal')
    # plt.show()

In [ ]:
def eqr_policy(s):
    """Equiprobable random policy.
    Returns a list of (action_index, probability) tuples.
    Note that s is unused since this is random policy
    """
    return [
        (0, 0.25),
        (1, 0.25),
        (2, 0.25),
        (3, 0.25),
    ]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
v_km1 = np.zeros(16)
v_k = np.zeros(16)

tol = 0.001
max_iter = 200
iter_ = 1

while iter_ <= max_iter:
    del_ = 0
    for s in range(16):
        for a, p_a_s in eqr_policy(s):
            for sp, r, p_spr_sa in dynamics(s, a):
                v_k[s] += p_a_s * p_spr_sa * (r + v_km1[sp])
        del_ = np.max([del_, np.abs(v_k[s] - v_km1[s])])
    v_km1 = v_k.copy()
    v_k = np.zeros(16)

    if del_ < tol:
        print(f"converged on iteration {iter_}")
        break

    iter_ += 1

In [ ]:
plot_v(v_km1)

In [ ]:
def calc_q_from_v(v):
    """Given a state-value function v and the problem dynamics,
    compute the corresponding action-value function.
    """
    gamma = 1
    v_f = v.flatten()
    q = np.zeros((16, 4)) # states, actions
    for s in range(16):
        for a in range(4):
            for sp, r, p_sp_r in dynamics(s, a):
                q[s, a] += (r + gamma * v_f[sp]) * p_sp_r
    return q


def calc_greedy_policy_from_v(v):
    """Given a state-value function, calculate a greedy policy."""
    # get action-value function
    q = calc_q_from_v(v)

    def greedy_policy(s):
        # Greedy policy assigns nonzero probability only to actions that
        # give maximal q(s,a)
        exp_return_a = [q[s,a] for a in range(4)]

        # then take the max
        opt_actions = np.where(exp_return_a == np.max(exp_return_a))[0]
        p_ = 1/len(opt_actions)
        return [(oa, p_) for oa in opt_actions]
    return greedy_policy

In [ ]:
q.shape

In [ ]:
policy_greedy = calc_greedy_policy_from_v(v_km1)

In [ ]:
plot_gridworld_policy(policy_greedy)

# Car example

State is $\left(s_1, s_2\right) \in \left[0, 20\right]^2$

Actions are $a \in \left[-5, 5\right]$

A policy is $ \pi\left(a\middle|s\right) $

The dynamics are given by

$$ \begin{align*} p_1\left(n_{req}\right) = \frac{3^n}{n!}\exp^{-3} \\ p_2\left(n_{req}\right) = \frac{4^n}{n!}\exp^{-4} \\ p_1\left(n_{ret}\right) = \frac{3^n}{n!}\exp^{-3} \\ p_2\left(n_{ret}\right) = \frac{2^n}{n!}\exp^{-2}   \end{align*} $$

In [ ]:
from math import factorial
def _poisson_pdf(lambda_, n):
    return (lambda_ ** n) * np.exp(-lambda_) / factorial(n)

def p1_req(n):
    """Probability of n requests from location 1."""
    return _poisson_pdf(3, n)
def p1_ret(n):
    """Probability of n returns to location 1."""
    return _poisson_pdf(3, n)
def p2_req(n):
    """Probability of n requests from location 2."""
    return _poisson_pdf(4, n)
def p2_ret(n):
    """Probability of n returns to location 2."""
    return _poisson_pdf(2, n)


actions = zip(range(11), range(-5,6,1))
def dynamics(s, a):
    """Return [(sp, r, p_spr_sa)]"""

    # loop over every scenario
    for n_req_1 in range(21):
        for n_req_2 in range(21):
            for n_ret_1 in range(21):
                for n_ret_2 in range(21):

                    # calculate the return (deterministic now)
                    r = 10 * np.max([0, s[0] - n_req_1]) + 10 * np.max([0, s[1] - n_req_2]) - 2 * actions[a]

                    # calculate the new state
                    s1 = (s[0] - np.max([0, s[0] - n_req_1])) - actions[a] + n_ret_1


In [ ]:
Z = np.ones((20, 20))
Z[:10, :10] = 10
Z[:5, :] = 5

fig, ax = plt.subplots()
img = ax.imshow(Z)
plt.colorbar(img)
plt.show()

The hard part here is calculating and storing the dynamics $p\left(s',r\middle|s,a\right)$.

$$ p\left(s_1', s_2', r \middle| s_1, s_2, a\right)$$

In [ ]:
from dataclasses import dataclass

@dataclass
class State:
    s1: int
    s2: int

@dataclass
class Action:
    a: int

states = dict(enumerate([ State(a,b) for a in range(0, 21) for b in range(0, 21) ]))
actions = dict(enumerate([ Action(a) for a in range(-5, 6)]))

Start by computing $p\left(s', r\middle| s,a\right)$.

In [ ]:
s = states[6]
a = actions[4]

# need to iterate over every possibility

if a.a > s.s1 or (a.a < 0 and -a.a > s.s2):
     prob_ = 0.0
else:
    for n_req_1 in range(21):
            p1_req_p = p1_req(n_req_1)
            for n_req_2 in range(21):
                p2_req_p = p2_req(n_req_2)
                for n_ret_1 in range(21):
                    p1_ret_p = p1_ret(n_ret_1)
                    for n_ret_2 in range(21):
                        p2_ret_p = p2_ret(n_ret_2)
                        # calculate the return (deterministic now)
                        r = 10 * np.max([0, s.s1 - n_req_1]) + 10 * np.max([0, s.s2 - n_req_2]) - 2 * a.a
                        
                        # calculate the new state
                        s1 = (
                             s.s1 - np.max([0, s.s1 - n_req_1]) # subtract demand
                            - a.a \ + n_ret_1
                        )
                        
                        # get the probability of this occurring


In [ ]:
a

# Start over

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import factorial

transition tuple is $\left(s, a, s', r, p\right)$

In [ ]:
transitions = []
states = [ (a, b) for a in range(21) for b in range(21) ]
# states = [ (a, b) for a in range(21) for b in range(21) ]

actions = dict(enumerate(range(-5, 6, 1)))


class Poisson:
    """Helper for obtaining probability distributions."""
    @staticmethod
    def _factorial(n_max):
        """Compute the factorials of an array of sequential integers."""
        n_fact = np.ones(n_max + 1, dtype=int)
        for i in range(2, n_max + 1):
            n_fact[i] = n_fact[i-1] * i
        return n_fact
    
    def get(lambda_):
        return (lambda_ ** np.arange(0,21,1)) * np.exp(-lambda_) / Poisson._factorial(20)
    
def action_valid(s, a):
    """Check that action a is valid in state s"""
    if a > 0:
        return a <= s[0] # valid if there are enough cars at loc A
    elif a < 0:
        return -a <= s[1] # valid if there are enough cars at loc B
    else:
        return True # a == 0

def calc_new_state_and_reward(s, a, n_a_ordered, n_a_returned, n_b_ordered, n_b_returned):
    """Compute s' and r for this combination of state, action, and observed customer
    behaviors.
    """
    # Move cars
    s_int = (s[0]-a, s[1]+a) # intermediate

    # Rent out as many cars as possible; accept returned cars
    # Clip total number of cars at 20
    n_a_rented = min([n_a_ordered, s_int[0]])
    n_b_rented = min([n_b_ordered, s_int[1]])
    sp = (
        np.max([s_int[0] - n_a_rented + n_a_returned, 20]),
        np.max([s_int[1] - n_b_rented + n_b_returned, 20]),
        )
    
    # Calculate the reward
    r = -2 * np.abs(a) + 10 * (n_a_rented + n_b_rented)
    return sp, r

p_a_order = p_a_return = Poisson.get(3)
p_b_order = Poisson.get(4)
p_b_return = Poisson.get(2)

# rather than construct a function like prob(s', r, s, a), which would be
# very intensive to construct and evaluate, I will create a list of transition
# tuples. Note that some future states and rewards (s', r, ...) will be repeated,
# corresponding to different samples from the stochastic dynamics. This is OK
# since we will be summing them anyway.
for s in states:
    for a_idx, a in actions.items():
        if action_valid(s, a):
            for n_a_ordered in range(2):
                for n_a_returned in range(2):
                    for n_b_ordered in range(2):
                        for n_b_returned in range(2):
            # for n_a_ordered in range(21):
            #     for n_a_returned in range(21):
            #         for n_b_ordered in range(21):
            #             for n_b_returned in range(21):
                            # calc probability of this happening (transition probability)
                            prob_customers = p_a_order[n_a_ordered] * p_a_return[n_a_returned] * p_b_order[n_b_ordered] * p_b_return[n_b_returned]

                            # calc new state and reward
                            sp, r = calc_new_state_and_reward(s=s, a=a, n_a_ordered=n_a_ordered, n_a_returned=n_a_returned, n_b_ordered=n_b_ordered, n_b_returned=n_b_returned)

                            # create transition tuple (s, a ; s', r, p)
                            transitions.append((s, a, sp, r, prob_customers))

\begin{align*}
 v_{k+1}\left(s\right) & = \sum_a \pi\left(a\middle|s\right) \sum_{s',r} p\left(s',r\middle|s,a\right) \left(r + \gamma v_{k}\left(s'\right) \right) \\
\end{align*}

In [ ]:
len(transitions)

# Start again

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class Poisson:
    """Helper for obtaining probability distributions."""
    @staticmethod
    def _factorial(n_max):
        """Compute the factorials of an array of sequential integers."""
        n_fact = np.ones(n_max + 1, dtype=int)
        for i in range(2, n_max + 1):
            n_fact[i] = n_fact[i-1] * i
        return n_fact
    
    def get(lambda_):
        return (lambda_ ** np.arange(0,21,1)) * np.exp(-lambda_) / Poisson._factorial(20)
    
# probs_customers[n_a_ordered, n_a_returned, n_b_ordered, n_b_returned]
p2 = Poisson.get(2)
p3 = Poisson.get(3)
p4 = Poisson.get(4)
probs_customers = np.zeros((21, 21, 21, 21))
for n_a_ordered in range(21):
    for n_a_returned in range(21):
        for n_b_ordered in range(21):
            for n_b_returned in range(21):
                probs_customers[n_a_ordered, n_a_returned, n_b_ordered, n_b_returned] = p3[n_a_ordered] * p3[n_a_returned] * p4[n_b_ordered] * p4[n_b_returned]

In [ ]:
s = (5, 6)
a = 3

def move_cars(s, a):
    """Allow impossible moves (these will be filtered out later)."""
    return (s[0] - a, s[1] + a)


def calculate_rewards(s_moved):
    """Calculate all possible rewards given a state and an action.
    There will be a different rewards for each different scenario, i.e.,
    combination of (n_a_ordered, n_a_returned, n_b_ordered, n_b_returned).
    """
    


# estimate the value of a state and action pair
s = (5, 6)
a = 3

s_moved = move_cars(s, a)




The update formula for policy evaluation is
$$ v_{k+1}\left(s\right) = \sum_{a} \pi\left(a\middle| s\right) \sum_{s', r} p\left(s', r \middle| s,a\right)\left[ r + \gamma v_k \left(s'\right)\right] $$

I am trying to rewrite this to make it simpler to compute. My issue now is that implementing the function $p\left(s', r \middle| s,a\right)$ is incredible complicated. It is hard to represent as an array because:
1. The states are actually tuples $\left(s_1, s_2\right)$, so the array would be six dimensional.
2. Worse, the rewards are a priori unknown and so it's impossible to know how big the reward dimension is and how to index it (since they are real numbers).

The first problem can be handled by standardizing an indexing scheme for the states. The second problem will be made irrelevant if I can remove $r$ as an index in these arrays.

To start, let's look at the innermost (double) summation:
$$ \sum_{s', r} p\left(s', r \middle| s,a\right)\left[ r + \gamma v_k \left(s'\right)\right] $$

I will split this out and try to manipulate it.

First, we can 
\begin{align*}
    p\left(s',r\middle|s,a\right) & = p\left(s'\middle|r,s,a\right) p\left(r\middle|s,a\right) \\
    & = p\left(s'\middle|s,a\right) p\left(r\middle|s,a\right) \\
\end{align*}
where the first equation comes from basic laws of conditional probability, and the second equation comes from conditional independence: $s'$ is conditionally independent of $r$ given $s,a$. Taking that inner sum above and substituting this factored form of the probability distribution, we get

\begin{align*}
    \sum_{s', r} p\left(s', r \middle| s,a\right)\left[ r + \gamma v_k \left(s'\right)\right] & = \sum_{s', r} r p\left(s', r \middle| s,a\right)+ \gamma \sum_{s', r} v_k \left(s'\right)p\left(s',r\middle|s,a\right) \\
    & = \sum_{s', r} r p\left(r\middle|s,a\right) p\left(s'\middle| s,a\right) + \gamma \sum_{s', r} v_k \left(s'\right)p\left(r\middle|s,a\right)p\left(s'\middle|s,a\right) \\
    & = \sum_{s'}  p\left(s'\middle| s,a\right) \sum_r r p\left(r\middle|s,a\right) + \gamma \sum_{s'} v_k \left(s'\right)p\left(s'\middle|s,a\right) \sum_r p\left(r\middle|s,a\right) \\
    & = \sum_r r p\left(r\middle|s,a\right) + \gamma \sum_{s'} v_k \left(s'\right)p\left(s'\middle|s,a\right) \\
\end{align*}

where the last equation follows from the definition of expected value and the law of total probability.

Finally, notice that the sums in the last line are actually expected values under the reward distribution and the transition distribution, respectively:
$$ \sum_{s',r} p\left(s',r\middle|s,a\right) \left[r + \gamma v_k\left(s'\right)\right] = \mathbb{E}\left[ R_{t+1} \middle| S_t=s, A_t=a \right] + \gamma \mathbb{E}\left[ v_k\left(S_{t+1}\right) \middle| S_t = s, A_t = a\right] $$

Does this help us solve the issues we were facing? I believe so! We should be able to precompute both terms and store them in a matrix, each indexed by $\left(s,a\right)$.

In the following, let a "scenario" be a particular instantiation of the random variables $\left(n_o^A, n_r^A, n_o^B, n_r^B\right)$, representing a possible combination of observed customer orders and returns. It is over these "scenarios" that the expectations are taken.

For notational convenience, let $S = \left\{0,1,\ldots,20\right\}^2$ be the state space and $A = \left\{-5,-4,\ldots, 4, 5\right\}$ be the action space.

#### The first term

$$ \mathbb{E}\left[ R_{t+1} \middle| S_t = s, A_t = a \right] $$

Given a state $s$ and an action $a$, we can store the expected reward function is a mapping $S \times A \rightarrow \mathbb{R}$. Since each state is two-dimensional, this could easily be a three-dimensional array; but since tuples are hashable in Python, I will first create a mapping from each state tuple to an integer index, then use this integer index in the reward expectation matrix.

I will precompute the expected reward for every $\left(s,a\right)$ by:
1. Generating every possible scenario $\left(n_o^A, n_r^A, n_o^B, n_r^B\right)$;
2. Generating the probability of each scenario;
3. Generating the reward $r$ the scenario would result in.

In [ ]:
MAX_NUM_CARS = 5

# probs_customers[n_a_ordered, n_a_returned, n_b_ordered, n_b_returned]
p2 = Poisson.get(2)
p3 = Poisson.get(3)
p4 = Poisson.get(4)
probs_customers = np.zeros((MAX_NUM_CARS+1,) * 4)
for n_a_ordered in range(MAX_NUM_CARS + 1):
    for n_a_returned in range(MAX_NUM_CARS + 1):
        for n_b_ordered in range(MAX_NUM_CARS + 1):
            for n_b_returned in range(MAX_NUM_CARS + 1):
                probs_customers[n_a_ordered, n_a_returned, n_b_ordered, n_b_returned] = p3[n_a_ordered] * p3[n_a_returned] * p4[n_b_ordered] * p4[n_b_returned]


aidx_to_action = dict(enumerate(range(-5, 6, 1)))
sidx_to_state = dict(enumerate([ (a, b) for a in range(0, MAX_NUM_CARS+1) for b in range(0, MAX_NUM_CARS+1)]) )
state_to_sidx = {v: k for k,v in sidx_to_state.items()}

n_states = len(state_to_sidx)
n_actions = len(aidx_to_action)

def get_n_scenarios():
    return (MAX_NUM_CARS+1) ** 4

def gen_get_scenarios():
    index = 0
    for n_o_A in range(0, MAX_NUM_CARS+1):
        for n_r_A in range(0, MAX_NUM_CARS+1):
            for n_o_B in range(0, MAX_NUM_CARS+1):
                for n_r_B in range(0, MAX_NUM_CARS+1):
                    yield (index, (n_o_A, n_r_A, n_o_B, n_r_B))
                    index += 1


def move_cars(s, a):
    """Move the cars. Does not validate!"""
    return (s[0] - a, s[1] + a)

def calc_new_state_and_reward(s, n_o_A, n_r_A, n_o_B, n_r_B):
    """Compute s' and r for this combination of state, action, and observed customer
    behaviors.
    """
    # Rent out as many cars as possible; accept returned cars
    # Clip total number of cars at 20
    n_rented_A = min([n_o_A, s[0]])
    n_rented_B = min([n_o_B, s[1]])
    sp = (
        np.min([s[0] - n_rented_A + n_r_A, MAX_NUM_CARS]),
        np.min([s[1] - n_rented_B + n_r_B, MAX_NUM_CARS]),
        )
    
    # Calculate the reward
    r = 10 * (n_rented_A + n_rented_B)
    return sp, r


e_r_sa = np.zeros((n_states, n_actions)) # trying to populate this
n_scenarios = get_n_scenarios()
for s_idx, state in sidx_to_state.items():
    for a_idx, action in aidx_to_action.items():

        # apply action
        state_moved = move_cars(state, action)

        # compute the expected reward for this state-action pair
        R = -2 * np.abs(action) * np.ones(n_scenarios)
        P = np.zeros(n_scenarios)
        for scen_idx, scenario in gen_get_scenarios():
            # get the probability of this scenario
            p_scenario = probs_customers[scenario]

            # get the reward and next state of this scenario
            state_prime, rs = calc_new_state_and_reward(state_moved, *scenario)
            # record probability or this scenario and the observed reward
            R[scen_idx] += rs
            P[scen_idx] = p_scenario

        # calc expected reward
        e_r_sa[s_idx, a_idx] = np.sum(R * P)


In [ ]:
sidx = state_to_sidx[(2,2)]
e_r_sa[sidx,:]

I'm computing reward for impossible state-action pairs: like $a=4$ for $s=\left(2,2\right)$. This should be OK, since any policy will have $\pi\left(a\middle|s\right) = 0$, thus this entry will be multiplied by zero when I compute the full sum.

#### The second term

$$ \mathbb{E}\left[ v_k\left(S_{t+1}\right) \middle| S_t = s, A_t = a\right] = \sum_{s'} v_k\left(s'\right) p\left(s' \middle| s, a\right)$$

Since this term depends on the current estimate $v_k$ of the state-value function, it can't be completely precomputed. However, we can compute the second distribution on the right, $p\left(s'\middle|s,a\right)$. This will be an $S\times S \times A$ matrix. In the code, I will call it the transition probability matrix.


In [ ]:
from tqdm import tqdm
from joblib import Parallel, delayed

In [287]:
MAX_NUM_CARS = 20

def get_scenario_probs():
    # probs_customers[n_a_ordered, n_a_returned, n_b_ordered, n_b_returned]
    p2 = Poisson.get(2)
    p3 = Poisson.get(3)
    p4 = Poisson.get(4)
    probs_customers = np.zeros((MAX_NUM_CARS+1,) * 4)
    for n_a_ordered in range(MAX_NUM_CARS + 1):
        for n_a_returned in range(MAX_NUM_CARS + 1):
            for n_b_ordered in range(MAX_NUM_CARS + 1):
                for n_b_returned in range(MAX_NUM_CARS + 1):
                    probs_customers[n_a_ordered, n_a_returned, n_b_ordered, n_b_returned] = p3[n_a_ordered] * p3[n_a_returned] * p4[n_b_ordered] * p2[n_b_returned]
    return probs_customers


aidx_to_action = dict(enumerate(range(-MAX_NUM_CARS, MAX_NUM_CARS+1, 1)))
action_to_aidx = {v: k for k,v in aidx_to_action.items()}
sidx_to_state = dict(enumerate([ (a, b) for a in range(0, MAX_NUM_CARS+1) for b in range(0, MAX_NUM_CARS+1) ]))
state_to_sidx = {v: k for k,v in sidx_to_state.items()}

n_states = len(state_to_sidx)
n_actions = len(aidx_to_action)

def get_n_scenarios():
    return (MAX_NUM_CARS+1) ** 4

def gen_get_scenarios():
    index = 0
    for n_o_A in range(0, MAX_NUM_CARS+1):
        for n_r_A in range(0, MAX_NUM_CARS+1):
            for n_o_B in range(0, MAX_NUM_CARS+1):
                for n_r_B in range(0, MAX_NUM_CARS+1):
                    yield (index, (n_o_A, n_r_A, n_o_B, n_r_B))
                    index += 1

def action_valid(s, a):
    """Check that action a is valid in state s"""
    if a > 0:
        return a <= s[0] # valid if there are enough cars at loc A
    elif a < 0:
        return -a <= s[1] # valid if there are enough cars at loc B
    else:
        return True # a == 0

def move_cars(s, a):
    """Move the cars. Does not validate!"""
    return (s[0] - a, s[1] + a)

def calc_new_state_and_reward(s, n_o_A, n_r_A, n_o_B, n_r_B):
    """Compute s' and r for this combination of state, action, and observed customer
    behaviors.
    """
    # Rent out as many cars as possible; accept returned cars
    # Clip total number of cars at 20
    n_rented_A = min([n_o_A, s[0]])
    n_rented_B = min([n_o_B, s[1]])
    sp = (
        np.min([s[0] - n_rented_A + n_r_A, MAX_NUM_CARS]),
        np.min([s[1] - n_rented_B + n_r_B, MAX_NUM_CARS]),
        )
    # Calculate the reward
    r = 10 * (n_rented_A + n_rented_B)
    return sp, r


def get_scenario_processor(state_moved):
    def process_scenario(scen_idx, scenario):
        # get the probability of this scenario
        p_scenario = probs_customers[scenario]

        # get the reward and next state of this scenario
        state_prime, rs = calc_new_state_and_reward(state_moved, *scenario)

        # get the transition probability for this scenario
        sp_index = state_to_sidx[state_prime] # new state
        # return stuff
        return sp_index, rs, p_scenario

    return process_scenario

e_r_sa = np.zeros((n_states, n_actions)) # trying to populate this
p_transitions = np.zeros((n_states, n_states, n_actions)) # trying to populate this
# (new_state, old_state, action)

n_scenarios = get_n_scenarios()
for s_idx, state in tqdm(sidx_to_state.items()):
    for a_idx, action in aidx_to_action.items():
        if not action_valid(state, action):
            e_r_sa[s_idx, a_idx] = 0
            p_transitions[:, s_idx, a_idx] = 0
            continue
        else:
            # apply action
            state_moved = move_cars(state, action)

            # compute the expected reward for this state-action pair
            # apply parallel here
            scenario_processor = get_scenario_processor(state_moved)
            
            results = Parallel(n_jobs=7)(delayed(scenario_processor)(scen[0], scen[1]) for scen in gen_get_scenarios())
            
            sp_indices = [r[0] for r in results] # s' (indices)
            res_arr = np.array(results)[:,1:] # [reward, probability]
            res_arr[:,1] += -2 * np.abs(action) # add moving cost

            p_transitions[sp_indices, s_idx, a_idx] += res_arr[:,1] # add the probability that this happens
            e_r_sa[s_idx, a_idx] = np.sum(res_arr[:,0] * res_arr[:,1]) # compute the expected reward

100%|██████████| 441/441 [4:30:36<00:00, 36.82s/it]  


In [289]:
p_transitions

array([[[  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        ...,
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]],

       [[  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        ...,
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]],

       [[  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        ...,
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]],

       ...,

       [[  0.,   0.,   0

In [280]:
# results # [(sp_index, rs, p_scenario)]
sp_indices = [r[0] for r in results]

results = np.array(results)


In [ ]:
scenario = (0,0,0,0)

# get the probability of this scenario
p_scenario = probs_customers[scenario]

# get the reward and next state of this scenario
state_moved = (1, )
state_prime, rs = calc_new_state_and_reward(state_moved, *scenario)
# # record probability of this scenario and the observed reward
# # R[scen_idx] += rs
# P[scen_idx] = p_scenario

# # record the transition probability for this scenario
# sp_index = state_to_sidx[state_prime] # new state
# p_transitions[sp_index, s_idx, a_idx] += p_scenario
# # return stuff

Can I parallelize this to speed things up?

In [ ]:
next(gen_get_scenarios())

\begin{align*}
 v_{k+1}\left(s\right) & = \sum_a \pi\left(a\middle|s\right) \sum_{s',r} p\left(s',r\middle|s,a\right) \left(r + \gamma v_{k}\left(s'\right) \right) = \sum_a \pi\left(a\middle|s\right) \left[\mathbb{E}\left[ R_{t+1} \middle| S_t=s, A_t=a \right] + \gamma \mathbb{E}\left[ v_k\left(S_{t+1}\right) \middle| S_t = s, A_t = a\right] \right]
\end{align*}

In [ ]:
# initial policy: never move cars
policy = np.zeros((n_actions, n_states))
policy[action_to_aidx[0], :] = 1.0

gamma = 0.9

v_old = np.zeros(len(sidx_to_state))
v_new = np.zeros_like(v)
for _ in range(5):
    # calculate the expected value of subsequent states, under the current value estimate, and given (s,a)
    for s_idx, state in sidx_to_state.items():
        for a_idx, action in aidx_to_action.items():
            v_new[s_idx] += policy[a_idx, s_idx] * (e_r_sa[s_idx, a_idx] + gamma * (v * p_transitions[:, s_idx, a_idx]).sum())

    v_old = v_new

In [ ]:
v_new

In [ ]:
p_transitions[:, s_idx, a_idx].sum()

In [ ]:
v.flatten() * p_transitions[:, s_idx, a_idx]

In [ ]:
# estimate the state-value function of this policy
v_0 = np.zeros(len(sidx_to_state)) # initial estimate